In [1]:
import numpy as np
import scipy.sparse as sps
import porepy as pp

In [2]:
import sys
sys.path.append(".")
from Transport_class import Transport
from Reaction_class import Reaction

/home/dell/.local/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


In [3]:
Nx=100
#Ny=10
phys_dims=[1]
#phys_dims=[1,1]
#g=pp.CartGrid([Nx,Ny],phys_dims)
g=pp.CartGrid([Nx],phys_dims)
g.compute_geometry()  

In [4]:
unity = np.ones(g.num_cells)
empty = np.empty(0)
porosity=unity
aperture=1
bc_type_Ca=["dir","dir"]
bc_value_Ca=[0.,1.]
bc_type_CaSiO3=["dir","dir"]
bc_value_CaSiO3=[0.,1.]
init_cond_Ca=lambda x,y,z:int(x<0.5)
init_cond_CaSiO3=lambda x,y,z:int(x<0.5)

In [5]:
reaction_data={"temperature":423,
       "A":1.9e-2,
       "rate_const":0.004466,
       "E":4700,
       "R":8.314,
       "K_eq":10e9,
       "ph":3.5}

In [6]:
reaction=Reaction(g,reaction_data)

In [7]:
data_reagent_Ca = {"bc_type": bc_type_Ca,
            "bc_value": bc_value_Ca,
            "time_step": 0.01,
            "mass_weight": porosity * aperture,
            "darcy_flux":np.ones(Nx+1),
            "t_max": 1.0,
            "method": "Explicit",
            "lambda_lin_decay":0,
            "initial_cond":init_cond_Ca}

In [8]:
data_reagent_CaSiO3= {"bc_type": bc_type_CaSiO3,
            "bc_value": bc_value_CaSiO3,
            "time_step": 0.01,
            "mass_weight": porosity * aperture,
            "darcy_flux":np.ones(Nx+1),
            "t_max": 1.0,
            "method": "Explicit",
            "lambda_lin_decay":0,
            "initial_cond":init_cond_CaSiO3}

In [9]:
Ca=np.zeros(g.num_cells)
Ca_transport=Transport(g,data_reagent_Ca)
Ca_transport.set_bc()
Ca_transport.set_initial_cond(Ca)
Ca_lhs,Ca_rhs_b,Ca_rhs_matrix=Ca_transport.get_transport_lhs_rhs()

In [18]:
IEsolver = sps.linalg.factorized(lhs)

data_transport=transport.data[pp.PARAMETERS]["transport"]
n_steps = int(np.round(data_transport["t_max"] / data_transport["time_step"]))

save_every=1

# Exporter
exporter = pp.Exporter(transport.g, file_name="tracer",folder_name="solution")
    
for i in range(n_steps):
    if np.isclose(i % save_every, 0):
        # Export existing solution (final export is taken care of below)
        exporter.write_vtu({"tracer":tracer}, time_step=int(i // save_every))
        if data_transport["method"]=="Explicit":
            tracer = IEsolver(rhs_matrix*tracer+rhs_b)
        else:
            tracer = IEsolver(rhs_matrix*tracer+rhs_b)
        
exporter.write_vtu({"tracer":tracer}, time_step=(n_steps // save_every))
time_steps = np.arange(0,data_transport["t_max"] + data_transport["time_step"], save_every * data_transport["time_step"])
exporter.write_pvd(time_steps)        

NameError: name 'transport' is not defined